In [1]:
import numpy as np
import torch
import os
import torch.nn as nn
from scipy.optimize import least_squares
import setenv
from matplotlib import pyplot as plt
from model import Net_SDE,Net_SDE_Pro
from generate import heston


In [2]:
device='cpu'
torch.set_default_tensor_type('torch.FloatTensor')

path = os.path.abspath(os.getcwd())
model_path = path + "/result_model/"
picture_path = path + "/result_picture/"
data_path = path + "/data/"
loss_path = path + "/result_loss/"

In [3]:
model_pro = torch.load(model_path+"ex1_nsde.pth")

In [4]:
for layer in model_pro.modules():
    if isinstance(layer, torch.nn.Linear):
        print(layer.weight)

Parameter containing:
tensor([[ 0.0000,  0.0000,  0.0000],
        [-0.0057, -0.0066, -0.0066],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [-0.0008, -0.0009, -0.0012],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [-0.0037, -0.0036, -0.0050],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [-0.0052, -0.0055, -0.0056],
        [ 0.0000,  0.0000,  0.0000]], requires_grad=True)
Parameter containing:
tensor([[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.000

In [5]:

ITM_call= torch.Tensor(torch.load(data_path+'Call_ITM_VG_train.pt')).to(device=device)
ITM_put= torch.Tensor(torch.load(data_path+'Put_ITM_VG_train.pt')).to(device=device)
OTM_call= torch.Tensor(torch.load(data_path+'Call_OTM_VG_train.pt')).to(device=device)
OTM_put = torch.Tensor(torch.load(data_path+'Put_OTM_VG_train.pt')).to(device=device)

In [6]:
ITM_call

tensor([[40.1165, 35.1269, 30.1374, 25.1485, 20.1629, 15.1896, 10.2628,  5.5005,
          1.3983],
        [40.2236, 35.2445, 30.2660, 25.2897, 20.3215, 15.3841, 10.5403,  5.9771,
          2.1960],
        [40.3499, 35.3816, 30.4146, 25.4522, 20.5048, 15.6059, 10.8466,  6.4375,
          2.8209],
        [40.4799, 35.5222, 30.5666, 25.6187, 20.6946, 15.8386, 11.1585,  6.8705,
          3.3671],
        [40.6241, 35.6772, 30.7337, 25.8018, 20.9025, 16.0922, 11.4825,  7.2886,
          3.8563],
        [40.7548, 35.8187, 30.8875, 25.9720, 21.0982, 16.3299, 11.7860,  7.6764,
          4.3032],
        [40.9964, 36.0827, 31.1780, 26.2991, 21.4787, 16.7884, 12.3617,  8.3920,
          5.1086],
        [41.1074, 36.2055, 31.3154, 26.4551, 21.6631, 17.0152, 12.6443,  8.7321,
          5.4840],
        [41.2132, 36.3231, 31.4469, 26.6046, 21.8407, 17.2338, 12.9151,  9.0559,
          5.8349],
        [41.4528, 36.5865, 31.7390, 26.9371, 22.2315, 17.7057, 13.4794,  9.7002,
          6.5225]])

In [7]:
OTM_call

tensor([[1.3983e+00, 1.1105e-01, 1.4241e-02, 2.0552e-03, 2.9325e-04, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00],
        [2.1960e+00, 3.7199e-01, 6.1894e-02, 1.1334e-02, 1.9728e-03, 2.1543e-04,
         8.4802e-06, 0.0000e+00, 0.0000e+00],
        [2.8209e+00, 7.3629e-01, 1.5001e-01, 2.9984e-02, 6.0726e-03, 1.2268e-03,
         2.4187e-04, 1.1019e-04, 6.0504e-05],
        [3.3671e+00, 1.1598e+00, 3.0049e-01, 7.1979e-02, 1.6994e-02, 3.7612e-03,
         7.9246e-04, 2.5447e-04, 1.3923e-04],
        [3.8563e+00, 1.5673e+00, 4.8852e-01, 1.3358e-01, 3.4835e-02, 8.8858e-03,
         1.9998e-03, 4.2771e-04, 1.5800e-04],
        [4.3032e+00, 1.9648e+00, 7.1607e-01, 2.2283e-01, 6.4679e-02, 1.7622e-02,
         4.6279e-03, 1.2000e-03, 4.1947e-04],
        [5.1086e+00, 2.7040e+00, 1.2229e+00, 4.7894e-01, 1.7175e-01, 5.9124e-02,
         2.0183e-02, 6.7001e-03, 2.3141e-03],
        [5.4840e+00, 3.0585e+00, 1.4929e+00, 6.4119e-01, 2.5175e-01, 9.3251e-02,
         3.3338e-02, 1.1986e-0